# Word Embeddings

In [1]:
from time import time
import gensim
from gensim.models import Word2Vec
from nltk.corpus import brown
from nltk import pos_tag
from nltk import word_tokenize
import json
from stanfordcorenlp import StanfordCoreNLP

In [2]:
# Locate the pruned word2vec sample in NLTK
from nltk.data import find
word2vec_sample = str(find('models/word2vec_sample/pruned.word2vec.txt'))

In [3]:
#start = time()
model = gensim.models.KeyedVectors.load_word2vec_format(word2vec_sample, binary=False) # load in word2vec format
#print(time()-start)

In [4]:
# Get the model's vocabulary
vocab = set(model.vocab)

In [5]:
len(vocab)

43981

In [6]:
# Note: words may have to be converted between lower and title case (and possibly upper case?) to see if they exist in the vocabulary
len(model['avocado']) # dimensions of vector representing word 'avocado' (and all other words)

300

In [7]:
model.most_similar('wagon', topn=10)

[('wagons', 0.6923391819000244),
 ('buckboard', 0.5725744366645813),
 ('tractor', 0.5646649599075317),
 ('truck', 0.5361799597740173),
 ('boxcar', 0.5161231756210327),
 ('Wagon', 0.5161173939704895),
 ('stagecoach', 0.5105308890342712),
 ('van', 0.5091939568519592),
 ('boxcars', 0.508715808391571),
 ('mule', 0.49818965792655945)]

In [8]:
# most similar using cosine similarity
model.most_similar_cosmul('wagon', topn=15)

[('wagons', 0.8461687564849854),
 ('buckboard', 0.7862864136695862),
 ('tractor', 0.7823317050933838),
 ('truck', 0.7680892944335938),
 ('boxcar', 0.758060872554779),
 ('Wagon', 0.7580579519271851),
 ('stagecoach', 0.7552647590637207),
 ('van', 0.7545962333679199),
 ('boxcars', 0.7543572187423706),
 ('mule', 0.7490941286087036),
 ('oxcart', 0.7466381192207336),
 ('chariot', 0.7398384809494019),
 ('buggy', 0.7397239208221436),
 ('locomotive', 0.7383973002433777),
 ('cart', 0.7375895977020264)]

In [9]:
# select most "unlike" item
model.doesnt_match(['guitar', 'trumpet', 'violin', 'flute'])

/home/loki/.local/lib/python3.7/site-packages/gensim/models/keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'trumpet'

In [10]:
# Capitalization and pluralization can lead to different most_similar results
for w in ('room', 'rooms', 'Room', 'Rooms'):
    print(w,'\t\n', model.most_similar(w),sep="")
    print('-'*50)

room	
[('rooms', 0.7605786323547363), ('upstairs', 0.6226500868797302), ('hallway', 0.6086891889572144), ('downstairs', 0.593078076839447), ('bathroom', 0.5513426065444946), ('kitchenette', 0.5502405166625977), ('basement', 0.5274615287780762), ('lounge', 0.5249817967414856), ('foyer', 0.5173395276069641), ('hallways', 0.5161101222038269)]
--------------------------------------------------
rooms	
[('room', 0.7605786323547363), ('Rooms', 0.634871780872345), ('bedrooms', 0.6270223259925842), ('bathrooms', 0.6007975339889526), ('suites', 0.5960015058517456), ('beds', 0.5761682987213135), ('lounges', 0.5643664598464966), ('kitchenette', 0.561974048614502), ('lounge', 0.5320551991462708), ('floors', 0.529586672782898)]
--------------------------------------------------
Room	
[('Rooms', 0.6796290278434753), ('Ballroom', 0.6294605731964111), ('Lounge', 0.5510711669921875), ('Auditorium', 0.5282275080680847), ('Building', 0.49195849895477295), ('Cafeteria', 0.49163320660591125), ('room', 0.485

In [11]:
# most_similar takes collections of vectors to be added (positive) or subtracted (negative) 
# Can use to specify an analogy: Read as "Paris is to France as Madrid is to ?""
model.most_similar(positive=['France', 'Madrid'], negative=['Paris'], topn=10)

[('Spain', 0.7776165008544922),
 ('Portugal', 0.6343989968299866),
 ('Argentina', 0.5653746724128723),
 ('Spanish', 0.5568934679031372),
 ('Porto', 0.5294696092605591),
 ('Italy', 0.5196953415870667),
 ('Brazil', 0.5041937828063965),
 ('Chile', 0.5033072233200073),
 ('Portuguese', 0.5029925107955933),
 ('Uruguay', 0.5019488334655762)]

In [12]:
# cosine similarity between two words
model.similarity('bolt','bread')

0.09666148

In [13]:
# cosine similarity between two sets of words
model.n_similarity(['Elena','bought','the','hat','today'], ['He','rode','his','red','wagon'])

0.42493808

In [14]:
model.similar_by_word('roadster',topn=15)

[('coupe', 0.8324612379074097),
 ('sedan', 0.6679246425628662),
 ('Bugatti', 0.6121565699577332),
 ('coachwork', 0.6103246212005615),
 ('sportiest', 0.60999596118927),
 ('sedans', 0.5992715954780579),
 ('Mustang', 0.5988028645515442),
 ('runabout', 0.5713080167770386),
 ('styling', 0.5562582612037659),
 ('hotrod', 0.5548204183578491),
 ('Jaguar', 0.5505430102348328),
 ('Giulietta', 0.5487755537033081),
 ('Volkswagens', 0.5426846742630005),
 ('streamliner', 0.5373216867446899),
 ('Chevy', 0.5369911789894104)]

In [15]:
model.similar_by_vector(model['roadster'],topn=15)

[('roadster', 1.0),
 ('coupe', 0.8324612379074097),
 ('sedan', 0.6679246425628662),
 ('Bugatti', 0.6121566295623779),
 ('coachwork', 0.6103246212005615),
 ('sportiest', 0.60999596118927),
 ('sedans', 0.5992715954780579),
 ('Mustang', 0.5988028645515442),
 ('runabout', 0.5713080167770386),
 ('styling', 0.5562582612037659),
 ('hotrod', 0.5548204183578491),
 ('Jaguar', 0.5505430102348328),
 ('Giulietta', 0.5487755537033081),
 ('Volkswagens', 0.5426846742630005),
 ('streamliner', 0.5373216867446899)]

### Training a custom Word2Vec model

In [16]:
# How-to train custom Word2Vec model (this is trained with the Brown corpus)
custommodel = Word2Vec(brown.sents(), size=300, window=5, min_count=20)

In [17]:
# To supress deprecation error in custom models or those loaded from saved files, prefix method call with "wv.""
# so to call the most_similar method, use
custommodel.wv.most_similar('wagon',topn=15)  # Note that the answers are much different from those of the pre-trained word2vec model

[('shoulders', 0.9721115827560425),
 ('seat', 0.9661572575569153),
 ('flying', 0.9656062126159668),
 ('holding', 0.9635875225067139),
 ('wind', 0.9608058333396912),
 ('coat', 0.9592040777206421),
 ('fingers', 0.9589619636535645),
 ('rifle', 0.9575096368789673),
 ('knocked', 0.9562466740608215),
 ('journey', 0.9559452533721924),
 ('beneath', 0.9558757543563843),
 ('wheel', 0.9556571245193481),
 ('knee', 0.9554955363273621),
 ('breath', 0.9546530246734619),
 ('shop', 0.9546340703964233)]

# Lemmatization

### Extracting lemmas with NLTK

In [18]:
# create an instance of the WordNet Lemmatizer
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [19]:
# The lemmatizer should be given a POS tag to return the lemma (defaults to nouns otherwise, and only works for simple cases like plurals)
# adjective = 'a'; adverb = 'r'
print(lemmatizer.lemmatize("sleeping",pos='v'))

sleep


### Extracting lemmas with StanfordCoreNLP (note: this will also return the part of speech tag)

In [ ]:
# Get an instance of StanfordCoreNLP by connecting to the server
#nlp = StanfordCoreNLP('http://jupyterlab-nfs-corenlp', port=9000)
nlp = StanfordCoreNLP('http://localhost', port=9000)

In [ ]:
txt = "The day was sunny and warm.  I decided I'd go boating."  # text to be annotated
props = {'annotators': 'lemma','outputFormat':'json'} # set annotator to provide lemma and get return as json (otherwise it's a string)
res = nlp.annotate(txt, properties=props)   # apply the annotator: results are in json format
d = json.loads(res)                         # load the json object into a dictionary

In [ ]:
# d is the dictionary returned from loading the json response
d

In [ ]:
# d['sentences'] is a list of dictionaries, one per sentence - so the second sentence (at index 1) is
first_sent_d = d['sentences'][1]

In [ ]:
# each dictionary has a key 'index' (value is integer sentence index) and a key 'tokens' (value is list of token dictionaries)
first_sent_d

In [ ]:
# each token dictionary contains (among other things) the original text, its corresponding lemma, and the POS tag:
for tok_d in first_sent_d['tokens']:
    print(tok_d['originalText'], tok_d['lemma'], tok_d['pos'])

# WordNet

In [ ]:
# import WordNet
from nltk.corpus import wordnet as wn

In [ ]:
wn.synsets('dog')  # Returns the synsets for the word "dog"

In [ ]:
wn.synsets('dog', pos='n')  # Restrict returned synsets to verbs (also works with NOUN, ADJ, ADV)

In [ ]:
# Select a specific synset
dog = wn.synset('dog.n.01')
dog.name() # dentifies the name of the synset of this variable

In [ ]:
dog.lemmas() # Outputs full lemmas, including part of speech and sense #

In [ ]:
dog.lemma_names() # Outputs lemma names

In [ ]:
dog.definition() # The gloss (or definition)

In [ ]:
dog.examples() # examples of use(s) in sentences

## Synonyms and Antonyms

In [ ]:
def syns_ants(word):
    synonyms = [] 
    antonyms = []      
    for syn in wn.synsets(word): 
        for l in syn.lemmas(): 
            synonyms.append(l.name()) 
            if l.antonyms(): 
                antonyms.append(l.antonyms()[0].name())

    synonyms = set(synonyms)
    antonyms = set(antonyms)
    return synonyms, antonyms

In [ ]:
syns, ants = syns_ants("calm")

print("Synonyms:",syns,"\n\nAntonyms:",ants,sep="")

## Hypernyms and Hyponyms

In [ ]:
dog.hypernyms() # Hypernyms for dog (e.g., a dog is-a ...)

In [ ]:
dog.hyponyms() # Hyponyms for dog (e.g., each of these is-a dog)

In [ ]:
dog.hypernym_paths() # Paths to all hypernyms (as returned by hypernyms method above)

In [ ]:
# Hypernym Tree
from pprint import pprint  # import pretty print 
hyp = lambda s:s.hypernyms() 
pprint(dog.tree(hyp)) # output hypernym tree 


In [ ]:
cat = wn.synset('cat.n.01') # get cat synset

In [ ]:
cat

In [ ]:
dog.common_hypernyms(cat) # what hypernyms are common to both dogs and cats

In [ ]:
# Lowest Common Hypernym between two synsets
wn.synset('hairdresser.n.01').lowest_common_hypernyms(wn.synset('chef.n.01'))

### Instance Hypernyms/Hyponyms

In [ ]:
city=wn.synset('city.n.01') 
city.instance_hyponyms()[:10] # Lists the first 10 hyponyms of city (individual cities)

In [ ]:
aca = wn.synset('acapulco.n.01')  # pick a city
aca.instance_hypernyms() # Lists all its hypernyms

## Holonyms and Meronyms

### Part Holonyms and Meronyms

In [ ]:
car=wn.synset('car.n.01') # get synset for car
car.part_meronyms()[:10]  # Returns top 10 entities that compose a car (accelerator, airbag, etc)

In [ ]:
accel = wn.synset('accelerator.n.01')  # get the synset for accelerator
accel.part_holonyms()                  # Returns holonyms for the synset: entities that the accelerator is a part-of

### Member Holonyms and Meronyms

In [ ]:
forest = wn.synset('forest.n.01')  # get synset for forest
forest.member_meronyms()           # Returns entities that are member-of a forest

In [ ]:
tree=wn.synset('tree.n.01')  # get the tree synset
tree.member_holonyms()       # Returns entities that the tree is a member-of

### Substance Holonyms and Meronyms

In [ ]:
bread= wn.synset('bread.n.01')  # get synset for bread
bread.substance_meronyms()      # Returns entities that are substances of bread

In [ ]:
flour = wn.synset('flour.n.01')  # get synset for flour
flour.substance_holonyms()       # Returns entities flour is a substances-of

## Similarity Measures

### Path-Based Similarities

In [ ]:
love = wn.synset('love.n.01')
romance = wn.synset('romance.n.01')
hate = wn.synset('hate.n.01')

**Path Similarity** (returns 0 to 1)

In [ ]:
# similarity is not synonymy
print('Love - Romance',love.path_similarity(romance))
print('Love - Hate',love.path_similarity(hate))
print('Romance - Hate',romance.path_similarity(hate))

**Leacock-Chodorow Similarity**

In [ ]:
print('Love - Romance',love.lch_similarity(romance))
print('Love - Hate',love.lch_similarity(hate))
print('Romance - Hate',romance.lch_similarity(hate))

**Wu-Palmer Similarity** (returns 0 to 1)

In [ ]:
print('Love - Romance',love.wup_similarity(romance))
print('Love - Hate',love.wup_similarity(hate))
print('Romance - Hate',romance.wup_similarity(hate))

### Information-Content Based Similarities

In [ ]:
from nltk.corpus import wordnet_ic as wic  # import that allows loading of information content
ic = wic.ic('ic-brown.dat')                # load information content from the Brown corpus int variable

**Lin Similarity**

In [ ]:
print('Love - Romance',love.lin_similarity(romance, ic))
print('Love - Hate',love.lin_similarity(hate, ic))
print('Romance - Hate',romance.lin_similarity(hate, ic))

**Resnik Similarity**

In [ ]:
print('Love - Romance',love.res_similarity(romance, ic))
print('Love - Hate',love.res_similarity(hate, ic))
print('Romance - Hate',romance.res_similarity(hate, ic))

**Jiang-Conrath Similarity**

In [ ]:
print('Love - Romance',love.jcn_similarity(romance, ic))
print('Love - Hate',love.jcn_similarity(hate, ic))
print('Romance - Hate',romance.jcn_similarity(hate, ic))